In [9]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/orgspec-epitope-prediction')  

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [10]:
import os
for dirname, _, filenames in os.walk('./input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./input/training-Ovolvulus.csv


In [11]:
import pandas as pd 

# Importing the dataset
train_ovolvulus = pd.read_csv('./input/training-Ovolvulus.csv')


# Only the b cell dataset is being used on this notebook for training and the covid dataset for testing (the sars dataset is not used for training)
train_ovolvulus.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Info_epitope_id,Info_sourceOrg_id,Info_protein_id,Info_host_id,Info_n_Positive,Info_n_Negative,Info_TSeq_accver,Info_TSeq_taxid,Info_TSeq_orgname,Info_center_pos,Info_window_seq,Class,feat_seq_entropy,feat_C_atoms,feat_H_atoms,feat_N_atoms,feat_O_atoms,feat_S_atoms,feat_molecular_weight,feat_Perc_Tiny,feat_Perc_Small,feat_Perc_Aliphatic,feat_Perc_Aromatic,feat_Perc_NonPolar,feat_Perc_Polar,feat_Perc_Charged,feat_Perc_Basic,feat_Perc_Acidic,feat_PP1,feat_PP2,feat_PP3,feat_KF1,feat_KF2,feat_KF3,feat_KF4,feat_KF5,feat_KF6,feat_KF7,feat_KF8,feat_KF9,...,feat_Perc_WA,feat_Perc_WC,feat_Perc_WD,feat_Perc_WE,feat_Perc_WF,feat_Perc_WG,feat_Perc_WH,feat_Perc_WI,feat_Perc_WK,feat_Perc_WL,feat_Perc_WM,feat_Perc_WN,feat_Perc_WP,feat_Perc_WQ,feat_Perc_WR,feat_Perc_WS,feat_Perc_WT,feat_Perc_WV,feat_Perc_WW,feat_Perc_WY,feat_Perc_YA,feat_Perc_YC,feat_Perc_YD,feat_Perc_YE,feat_Perc_YF,feat_Perc_YG,feat_Perc_YH,feat_Perc_YI,feat_Perc_YK,feat_Perc_YL,feat_Perc_YM,feat_Perc_YN,feat_Perc_YP,feat_Perc_YQ,feat_Perc_YR,feat_Perc_YS,feat_Perc_YT,feat_Perc_YV,feat_Perc_YW,feat_Perc_YY
0,854007,6282,A0A044QLL7,"9606,9606",1,1,NaN,6282,Onchocerca volvulus,1,LLLLLLLLDVHIESG,1,2.306891,79,165,17,35,0,1913.23,0.133333,0.266667,0.666667,0.066667,0.733333,0.266667,0.200000,0.066667,0.133333,-0.466667,-0.240000,-0.186667,-0.586667,-0.228000,-0.108667,-0.496667,-0.344667,-1.077333,0.125333,-0.198667,0.180667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,854007,6282,A0A044QLL7,"9606,9606",1,1,NaN,6282,Onchocerca volvulus,2,LLLLLLLDVHIESGE,1,2.463458,78,161,17,37,0,1929.19,0.133333,0.266667,0.600000,0.066667,0.666667,0.333333,0.266667,0.066667,0.200000,-0.344000,-0.278000,-0.236667,-0.614000,-0.215333,-0.200000,-0.345333,-0.395333,-0.914000,0.064000,-0.122667,0.127333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,854007,6282,A0A044QLL7,"9606,9606",1,1,NaN,6282,Onchocerca volvulus,3,LLLLLLDVHIESGEV,1,2.606239,77,159,17,37,0,1915.17,0.133333,0.333333,0.600000,0.066667,0.666667,0.333333,0.266667,0.066667,0.200000,-0.350667,-0.308667,-0.230000,-0.594000,-0.262667,-0.048000,-0.298667,-0.325333,-0.831333,-0.071333,-0.068000,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,854007,6282,A0A044QLL7,"9606,9606",1,1,NaN,6282,Onchocerca volvulus,4,LLLLLDVHIESGEVC,1,2.866248,74,153,17,37,1,1905.16,0.200000,0.400000,0.533333,0.066667,0.666667,0.333333,0.266667,0.066667,0.200000,-0.327333,-0.342000,-0.201333,-0.516667,-0.322000,-0.002000,-0.295333,-0.336000,-0.534000,-0.034000,-0.063333,0.112000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,854007,6282,A0A044QLL7,"9606,9606",1,1,NaN,6282,Onchocerca volvulus,5,LLLLDVHIESGEVCA,1,3.106891,71,147,17,37,1,1863.08,0.266667,0.466667,0.533333,0.066667,0.666667,0.333333,0.266667,0.066667,0.200000,-0.331333,-0.394667,-0.164667,-0.551333,-0.433333,-0.050667,-0.240000,-0.361333,-0.449333,-0.111333,-0.018000,0.096000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Using only 1/3 of the dataset due to high RAM consumption
train_ovolvulus = train_ovolvulus.iloc[0:int(train_ovolvulus.shape[0]/3),:]

In [13]:
train_ovolvulus.Class.value_counts()

 1    11232
-1    10978
Name: Class, dtype: int64

In [14]:
# Checking for null values. 
train_ovolvulus.isnull().sum()

Info_epitope_id      0
Info_sourceOrg_id    0
Info_protein_id      0
Info_host_id         0
Info_n_Positive      0
                    ..
feat_Perc_YS         0
feat_Perc_YT         0
feat_Perc_YV         0
feat_Perc_YW         0
feat_Perc_YY         0
Length: 857, dtype: int64

In [15]:
# Splitting using train_test_split form sklearn.model_selection

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

train_ovolvulus = shuffle(train_ovolvulus)

X = train_ovolvulus.drop(['Class'], axis = 1)
y = train_ovolvulus['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Transforming traning dataset
X_train = X_train.drop(["Info_epitope_id","Info_sourceOrg_id","Info_protein_id","Info_host_id","Info_n_Positive", "Info_n_Negative", "Info_TSeq_accver", "Info_TSeq_taxid", "Info_TSeq_orgname", "Info_center_pos", "Info_window_seq"],axis =1)


# Transforming testing dataset
X_test = X_test.drop(["Info_epitope_id","Info_sourceOrg_id","Info_protein_id","Info_host_id","Info_n_Positive", "Info_n_Negative", "Info_TSeq_accver", "Info_TSeq_taxid", "Info_TSeq_orgname", "Info_center_pos", "Info_window_seq"],axis =1)
                                              

In [16]:
# Splitting the dataset manually

train_pct_index = int(0.8 * len(X))
X_train, X_test = X[:train_pct_index], X[train_pct_index:]
y_train, y_test = y[:train_pct_index], y[train_pct_index:]


# Transforming traning dataset
X_train = X_train.drop(["Info_epitope_id","Info_sourceOrg_id","Info_protein_id","Info_host_id","Info_n_Positive", "Info_n_Negative", "Info_TSeq_accver", "Info_TSeq_taxid", "Info_TSeq_orgname", "Info_center_pos", "Info_window_seq"],axis =1)


# Transforming testing dataset
X_test = X_test.drop(["Info_epitope_id","Info_sourceOrg_id","Info_protein_id","Info_host_id","Info_n_Positive", "Info_n_Negative", "Info_TSeq_accver", "Info_TSeq_taxid", "Info_TSeq_orgname", "Info_center_pos", "Info_window_seq"],axis =1)


**As we can see, the two cells above are equivalent and have the same result. You can comment on one of them**

In [ ]:
!pip install yellowbrick==0.9.1 scikit-learn==0.22.2
!pip install imbalanced-learn

In [ ]:

!pip install mlrose
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install imblearn


In [19]:
#Defining the pipeline

from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler


pipeline = imbpipeline(steps = [['smote', SMOTE()],
                                ['scaler', MinMaxScaler()],
                                ['classifier', KNeighborsClassifier()]])

stratified_kfold = StratifiedKFold(n_splits=10)

In [20]:

# Evaluating the model using cross-validation without tune hyperparameters (training dataset)

from sklearn.model_selection import cross_val_score
import numpy as np

f1 = cross_val_score(pipeline, X_train, y_train, scoring='f1', cv=stratified_kfold, n_jobs=-1)
roc_auc = cross_val_score(pipeline, X_train, y_train, scoring='roc_auc', cv=stratified_kfold, n_jobs=-1)


print('F1:', np.mean(f1))
print('AUC:', np.mean(roc_auc))

F1: 0.7871681788156024
AUC: 0.8959302862949066


In [21]:
# Evaluating the model using cross-validation and tune hyperparameters (training dataset)

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")


param_grid = {'classifier__n_neighbors': range(1, 20)}


# Creating a dictionary with the metrics to be calculated
scores = {'accuracy' :make_scorer(accuracy_score),
               'recall'   :make_scorer(recall_score),
               'precision':make_scorer(precision_score),
               'f1'       :make_scorer(fbeta_score, beta = 1),
               'roc_auc'  :make_scorer(roc_auc_score) }


grid_search = GridSearchCV(estimator = pipeline,
                      param_grid = param_grid,
                      cv = stratified_kfold,
                      scoring = scores,   
                      refit = 'f1'
                      )           


grid_search.fit(X_train, y_train)

# Prints the column names that are stored in grid_search.
#pd.DataFrame(grid_search.cv_results_).columns.tolist()


result = pd.DataFrame(grid_search.cv_results_)[[
                                  'mean_test_recall',
                                  'mean_test_precision',
                                  'mean_test_f1',
                                  'mean_test_roc_auc'
                                  ]]

print(result.sort_values(ascending=False, by='mean_test_f1').head())




    mean_test_recall  mean_test_precision  mean_test_f1  mean_test_roc_auc
12          0.825723             0.811435      0.818453           0.814229
14          0.821502             0.813003      0.817153           0.813601
16          0.820393             0.813930      0.817083           0.813790
10          0.822500             0.810914      0.816582           0.812675
18          0.817393             0.815164      0.816221           0.813430


In [24]:

# Evaluating test set prediction without hyperparameter tune
pipeline.fit(X_train, y_train)

pred = pipeline.predict(X_test)
print("F1 : ",fbeta_score(y_test, pred, beta=0.5))
print("AUC: ", roc_auc_score(y_test,pred))


F1 :  0.7760411961333454
AUC:  0.7742132879714612


In [25]:
# Evaluating test set prediction with hyperparameter tune
pred=grid_search.predict(X_test)
print("F1 : ",fbeta_score(y_test, pred, beta=0.5))
print("AUC: ", roc_auc_score(y_test,pred))

F1 :  0.8148712889048344
AUC:  0.8142696294602412


As we can see above, tuning the hyperparameters improve the model's performance.